In [2]:
import pandas_datareader as web
import pandas as pd
import numpy as np
import collections
from db_connector import connect_db
from db_connector import engine_db
from sqlalchemy import create_engine
import pymysql

if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

engine = engine_db()
con = connect_db()

mycursor = con.cursor()

truncate_table = """
truncate table global_yield;
"""

mycursor.execute(truncate_table)
con.commit()

query_yeild = """
    insert into global_yield (Date, T10Y2Y, T10Y3M, T10YIE)
    values (%s,%s,%s,%s);
"""

t10y2y = web.DataReader('T10Y2Y', 'fred', start='2020-01-01')
t10y3m = web.DataReader('T10Y3M', 'fred', start='2020-01-01')
t10yie = web.DataReader('T10YIE', 'fred', start='2020-01-01')

rate_diff = pd.concat([t10y2y, t10y3m,t10yie], axis=1)
rate_diff.reset_index(inplace = True)
rate_diff = rate_diff.replace([np.nan], None)
rate_diff.columns = ['Date', 'T10Y2Y', 'T10Y3M', 'T10YIE']

args = rate_diff.values.tolist()
mycursor.executemany(query_yeild, args)
con.commit()

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime
import collections
from db_connector import connect_db
from db_connector import engine_db
from sqlalchemy import create_engine
import pymysql

if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

engine = engine_db()
con = connect_db()

mycursor = con.cursor()

fear_greed = """
    insert into fear_greed (Date, value)
    values (%s,%s) as new
    on duplicate key update
    value = new.value;
"""

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url='https://edition.cnn.com/markets/fear-and-greed')
idx = driver.find_element(By.CLASS_NAME,
                          value='market-fng-gauge__dial-number-value').text
driver.close()
idx = int(idx)

args = [datetime.today().strftime('%Y-%m-%d'),idx]
mycursor.execute(fear_greed, args)
con.commit()
